# Joint clustering of mouse liver data

The data for this project is sourced from the Gene Expression Omnibus (GEO) under accession number [GSE155182](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE155182). The raw sequencing files are associated with four samples:

1. **Normal Liver - Whole (GSM4696914):** Hepatic non-parenchymal cells from normal mouse liver.

2. **Normal Liver - Tomato Positive (GSM4696915):** Hepatic non-parenchymal cells enriched for p16-positive cells.

3. **NASH Liver - Whole (GSM4696916):** Hepatic non-parenchymal cells from NASH mouse liver.

4. **NASH Liver - Tomato Positive (GSM4696917):** Hepatic non-parenchymal cells enriched for p16-positive cells.

In [1]:
import sys
import scanpy as sc
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import anndata

### Preprocessing

In [3]:
# Make observation names unique for the specified objects
ad_normal_w = sc.read_10x_h5(filepath_normal_w)
ad_normal_w.obs_names_make_unique()

ad_normal_tpos = sc.read_10x_h5(filepath_normal_tpos)
ad_normal_tpos.obs_names_make_unique()

ad_nash_w = sc.read_10x_h5(filepath_nash_w)
ad_nash_w.obs_names_make_unique()

ad_nash_tpos = sc.read_10x_h5(filepath_nash_tpos)
ad_nash_tpos.obs_names_make_unique()

/home/nb752/.conda/envs/CellBenderEnv/lib/python3.7/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
def print_adata_info(adata, name):
    dimensions = adata.shape
    unique_genes = adata.var_names.nunique()
    print(f"{name}:")
    print(f"  Dimensions: {dimensions}")
    print(f"  Number of unique genes: {unique_genes}\n")
    return (name, dimensions, unique_genes)

# Print information for the new AnnData objects
info_normal_w = print_adata_info(ad_normal_w, 'ad_normal_w')
info_normal_tpos = print_adata_info(ad_normal_tpos, 'ad_normal_tpos')
info_nash_w = print_adata_info(ad_nash_w, 'ad_nash_w')
info_nash_tpos = print_adata_info(ad_nash_tpos, 'ad_nash_tpos')

ad_normal_w:
  Dimensions: (3885, 33696)
  Number of unique genes: 33670

ad_normal_tpos:
  Dimensions: (5120, 33696)
  Number of unique genes: 33670

ad_nash_w:
  Dimensions: (5345, 33696)
  Number of unique genes: 33670

ad_nash_tpos:
  Dimensions: (6438, 33696)
  Number of unique genes: 33670



In [5]:
# Function to remove lncRNAs
def remove_lncRNAs(adata):
    gene_names = adata.var_names
    mask = ~gene_names.str.match(r'Gm\d+')  # Exclude genes matching the lncRNA pattern
    adata = adata[:, mask]  # Keep only genes that do not match the pattern
    return adata

# Apply the function to remove lncRNAs
ad_normal_w = remove_lncRNAs(ad_normal_w)
ad_normal_tpos = remove_lncRNAs(ad_normal_tpos)
ad_nash_w = remove_lncRNAs(ad_nash_w)
ad_nash_tpos = remove_lncRNAs(ad_nash_tpos)

# Print information for the updated AnnData objects
info_normal_w = print_adata_info(ad_normal_w, 'ad_normal_w')
info_normal_tpos = print_adata_info(ad_normal_tpos, 'ad_normal_tpos')
info_nash_w = print_adata_info(ad_nash_w, 'ad_nash_w')
info_nash_tpos = print_adata_info(ad_nash_tpos, 'ad_nash_tpos')

ad_normal_w:
  Dimensions: (3885, 24263)
  Number of unique genes: 24247

ad_normal_tpos:
  Dimensions: (5120, 24263)
  Number of unique genes: 24247

ad_nash_w:
  Dimensions: (5345, 24263)
  Number of unique genes: 24247

ad_nash_tpos:
  Dimensions: (6438, 24263)
  Number of unique genes: 24247



In [6]:
# Find duplicated genes in each dataset
duplicated_normal_w = ad_normal_w.var_names[ad_normal_w.var_names.duplicated()]
duplicated_normal_tpos = ad_normal_tpos.var_names[ad_normal_tpos.var_names.duplicated()]
duplicated_nash_w = ad_nash_w.var_names[ad_nash_w.var_names.duplicated()]
duplicated_nash_tpos = ad_nash_tpos.var_names[ad_nash_tpos.var_names.duplicated()]

# Combine all duplicated genes into a single list
all_duplicated_genes = set(duplicated_normal_w).union(
    set(duplicated_normal_tpos),
    set(duplicated_nash_w),
    set(duplicated_nash_tpos)
)

duplicated_genes = list(all_duplicated_genes)

# Function to calculate and print gene sums
def print_gene_sums(ad, dataset_name):
    print(f"\nFor {dataset_name}")
    for gene in duplicated_genes:
        # Check if the gene exists in the dataset
        if gene in ad.var_names:
            gene_sum = ad[:, gene].X.sum(axis=0)  # Sums across all cells for the gene
        else:
            gene_sum = 0  # If gene is not present, use 0
        print(f"{gene}: {gene_sum}")

# Print gene sums for the new datasets
print_gene_sums(ad_normal_w, "ad_normal_w")
print_gene_sums(ad_normal_tpos, "ad_normal_tpos")
print_gene_sums(ad_nash_w, "ad_nash_w")
print_gene_sums(ad_nash_tpos, "ad_nash_tpos")


For ad_normal_w
Fam220a: [[ 39. 197.]]
Or5ae1: [[0. 0.]]
St6galnac2: [[31.  0.]]
Pakap: [[   31. 16673.]]
C730027H18Rik: [[0. 3.]]
Nnt: [[55.  0.]]
4933427D14Rik: [[19.  5.]]
Septin2: [[188.   0.]]
Lhb: [[0. 0.]]
Ptp4a1: [[41. 11.]]
Nrg1: [[192.   0.]]
Dpep2: [[35.  6.]]
Ddit3: [[999.   0.]]
Fam90a1b: [[1. 3.]]
Cdhr17: [[0. 0.]]
Aldoa: [[3.539e+03 2.000e+00]]

For ad_normal_tpos
Fam220a: [[ 34. 250.]]
Or5ae1: [[0. 0.]]
St6galnac2: [[33.  0.]]
Pakap: [[   79. 35788.]]
C730027H18Rik: [[1. 2.]]
Nnt: [[58.  0.]]
4933427D14Rik: [[29.  6.]]
Septin2: [[236.   0.]]
Lhb: [[0. 0.]]
Ptp4a1: [[49. 16.]]
Nrg1: [[851.   0.]]
Dpep2: [[13.  5.]]
Ddit3: [[1410.    0.]]
Fam90a1b: [[1. 8.]]
Cdhr17: [[0. 0.]]
Aldoa: [[4006.    6.]]

For ad_nash_w
Fam220a: [[ 73. 248.]]
Or5ae1: [[0. 0.]]
St6galnac2: [[32.  0.]]
Pakap: [[  222. 37870.]]
C730027H18Rik: [[ 0. 11.]]
Nnt: [[90.  0.]]
4933427D14Rik: [[55.  6.]]
Septin2: [[344.   0.]]
Lhb: [[0. 0.]]
Ptp4a1: [[48. 26.]]
Nrg1: [[1447.    0.]]
Dpep2: [[111.   7.]]


In [7]:
# Define duplicated genes
duplicated_genes = ['Ptp4a1', 'St6galnac2', 'Fam220a', '4933427D14Rik', 'Dpep2', 'C730027H18Rik', 'St6galnac2', 'Pakap', 'Aldoa']

duplicated_gene_info = ad_normal_w.var[ad_normal_w.var.index.isin(duplicated_genes)]
duplicated_gene_ids = duplicated_gene_info[['gene_ids']]

unique_gene_ids_df = duplicated_gene_ids.groupby(duplicated_gene_ids.index).agg({
    'gene_ids': lambda x: list(x)  # or ', '.join(x) if you want them as a comma-separated string
}).reset_index()


unique_gene_ids_df.columns = ['gene_name', 'ensembl_ids']
print(unique_gene_ids_df)

       gene_name                               ensembl_ids
0  4933427D14Rik  [ENSMUSG00000020807, ENSMUSG00000107877]
1          Aldoa  [ENSMUSG00000030695, ENSMUSG00000114515]
2  C730027H18Rik  [ENSMUSG00000112366, ENSMUSG00000112189]
3          Dpep2  [ENSMUSG00000053687, ENSMUSG00000115067]
4        Fam220a  [ENSMUSG00000118332, ENSMUSG00000083012]
5          Pakap  [ENSMUSG00000090053, ENSMUSG00000038729]
6         Ptp4a1  [ENSMUSG00000026064, ENSMUSG00000117310]
7     St6galnac2  [ENSMUSG00000057286, ENSMUSG00000110170]


In [8]:
# Define pairs of genes and gene_ids to remove
remove_pairs = [
    ('4933427D14Rik', 'ENSMUSG00000107877'), 
    ('Aldoa', 'ENSMUSG00000114515'),
    ('Dpep2', 'ENSMUSG00000115067'),
    ('Fam220a', 'ENSMUSG00000118332'),
    ('Pakap', 'ENSMUSG00000090053'),
    ('Ptp4a1', 'ENSMUSG00000117310'),
    ('C730027H18Rik', 'ENSMUSG00000112366'),
    ('St6galnac2', 'ENSMUSG00000110170')
]

remove_df = pd.DataFrame(remove_pairs, columns=['gene_name', 'gene_ids'])

# Function to remove specified genes and gene_ids from a dataset
def remove_genes(adata, remove_df):
    mask = ~adata.var.reset_index().set_index(['index', 'gene_ids']).index.isin(
        remove_df.set_index(['gene_name', 'gene_ids']).index
    )
    adata._inplace_subset_var(mask)

# Apply the function to your datasets
remove_genes(ad_normal_w, remove_df)
remove_genes(ad_normal_tpos, remove_df)
remove_genes(ad_nash_w, remove_df)
remove_genes(ad_nash_tpos, remove_df)

/home/nb752/.conda/envs/CellBenderEnv/lib/python3.7/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [9]:
# Define genes to remove
remove_genes = ['Nnt', 'Cdhr17', 'Lhb', 'Ddit3', 'Septin2', 'Or5ae1', 'Nrg1', 'Fam90a1b']

# Apply the mask to each dataset
mask = ~ad_normal_w.var.index.isin(remove_genes)
ad_normal_w._inplace_subset_var(mask)

mask = ~ad_normal_tpos.var.index.isin(remove_genes)
ad_normal_tpos._inplace_subset_var(mask)

mask = ~ad_nash_w.var.index.isin(remove_genes)
ad_nash_w._inplace_subset_var(mask)

mask = ~ad_nash_tpos.var.index.isin(remove_genes)
ad_nash_tpos._inplace_subset_var(mask)

In [10]:
ad_normal_w.var_names[ad_normal_w.var_names.duplicated()]

Index([], dtype='object')

### Joint clustering

In [11]:
# Add metadata for ad_normal_w
ad_normal_w.obs['treatment'] = 'normal'
ad_normal_w.obs['sample'] = 'whole'

# Add metadata for ad_normal_tpos
ad_normal_tpos.obs['treatment'] = 'normal'
ad_normal_tpos.obs['sample'] = 'T pos'

# Add metadata for ad_nash_w
ad_nash_w.obs['treatment'] = 'nash'
ad_nash_w.obs['sample'] = 'whole'

# Add metadata for ad_nash_tpos
ad_nash_tpos.obs['treatment'] = 'nash'
ad_nash_tpos.obs['sample'] = 'T pos'

In [12]:
# Concatenate normal liver and NASH liver datasets
adata_liver = anndata.concat([ad_normal_w, ad_nash_w], join='inner')

# Concatenate normal Tom+ and NASH Tom+ datasets
adata_tom = anndata.concat([ad_normal_tpos, ad_nash_tpos], join='inner')

/home/nb752/.conda/envs/CellBenderEnv/lib/python3.7/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [13]:
adata_tom

AnnData object with n_obs × n_vars = 11558 × 24239
    obs: 'treatment', 'sample'

In [14]:
adata_liver

AnnData object with n_obs × n_vars = 9230 × 24239
    obs: 'treatment', 'sample'

We have two separate AnnData objects now:

- **adata_liver**: Captures broader liver microenvironment changes between normal and NASH liver.
- **adata_tom**: Focuses on senescent (Tom+) cells for identifying condition-specific transcriptional changes related to NASH progression.